# The Stigler diet problem
Source: https://developers.google.com/optimization/lp/stigler_diet

A description of the problem can be found at https://en.wikipedia.org/wiki/Stigler_diet

In [1]:
from ortools.linear_solver import pywraplp
from dietdata import *

In [2]:
def main():
    solver = pywraplp.Solver.CreateSolver('GLOP')
    if not solver:
        return

    foods = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]    # Declare an array to hold our variables
    print('Number of variables =', solver.NumVariables())

    constraints = []    # Create the constraints, one per nutrient
    for i, nutrient in enumerate(nutrients):
        constraints.append(solver.Constraint(nutrient[1], solver.infinity()))
        for j, item in enumerate(data):
            constraints[i].SetCoefficient(foods[j], item[i + 3])
    print('Number of constraints =', solver.NumConstraints())

    objective = solver.Objective()    # Objective function: Minimize the sum of (price-normalized) foods
    for food in foods:
        objective.SetCoefficient(food, 1)
    objective.SetMinimization()

    status = solver.Solve()

    if status != solver.OPTIMAL:    # Check that the problem has an optimal solution
        print('The problem does not have an optimal solution!')
        if status == solver.FEASIBLE:
            print('A potentially suboptimal solution was found.')
        else:
            print('The solver could not solve the problem.')
            exit(1)

    nutrients_result = [0] * len(nutrients)    # Display the amounts (in dollars) to purchase of each food
    print('\nAnnual Foods')
    print('----------------------------------')
    for i, food in enumerate(foods):
        if food.solution_value() > 0.0:
            print('{:<25}  ${:>6.2f}'.format(data[i][0], 365. * food.solution_value()))
            for j, _ in enumerate(nutrients):
                nutrients_result[j] += data[i][j + 3] * food.solution_value()
    print('----------------------------------')
    print('Optimal annual price       ${:>6.2f}'.format(365. * objective.Value()))

    print('\nNutrients per day')
    print('--------------------------------------')
    for i, nutrient in enumerate(nutrients):
        print('{:<17}  {:>7.2f} (min {:>5.1f})'.format(nutrient[0], nutrients_result[i], nutrient[1]))

    print('\nAdvanced usage:')
    print('Problem solved in', solver.wall_time(), 'milliseconds')
    print('Problem solved in', solver.iterations(), 'iterations')

In [3]:
if __name__ == '__main__':
    main()

Number of variables = 77
Number of constraints = 9

Annual Foods
----------------------------------
Wheat Flour (Enriched)     $ 10.77
Liver (Beef)               $  0.69
Cabbage                    $  4.09
Spinach                    $  1.83
Navy Beans, Dried          $ 22.28
----------------------------------
Optimal annual price       $ 39.66

Nutrients per day
--------------------------------------
Calories (kcal)       3.00 (min   3.0)
Protein (g)         147.41 (min  70.0)
Calcium (g)           0.80 (min   0.8)
Iron (mg)            60.47 (min  12.0)
Vitamin A (KIU)       5.00 (min   5.0)
Vitamin B1 (mg)       4.12 (min   1.8)
Vitamin B2 (mg)       2.70 (min   2.7)
Niacin (mg)          27.32 (min  18.0)
Vitamin C (mg)       75.00 (min  75.0)

Advanced usage:
Problem solved in 1 milliseconds
Problem solved in 14 iterations
